In [1]:
import numpy as np
import pandas as pd
from numpy import *
from numpy import linalg as la
import scipy
from scipy import sparse
from scipy.sparse import *
from scipy.sparse.linalg import svds

In [2]:
data = []
with open('anonymous-msweb.data') as f:
    count = 0
    count_C = -1
    lis = []
    res = []
    for i in f.readlines():
        line = i.strip().split(',')
        data.append(line)
        dict_A = {}
        dict_C = {}
    for i in range(0,7):
        data.pop(0)
    for i in data:
        if i[0]=='A' and i[1] not in dict_A:
            dict_A[i[1]] = count
            count += 1
        if i[0]=='C':
            if len(lis) != 0: 
                res.append(lis)
            lis = []
            for j in range(len(dict_A)):
                lis.append(0.0)
            continue
        if i[0]=='V':
            lis[dict_A[i[1]]] += 1

In [3]:
res = np.mat(res)

In [ ]:
U,Sigma,VT=la.svd(res)

In [ ]:
from numpy import *
from numpy import linalg as la
# inA和inB都是列向量
def ecludSim(inA, inB) :
    return 1.0/(1.0 + la.norm(inA - inB))

def pearsSim(inA, inB) :
    # 检查是否存在三个或更多的点，若不存在，则返回1.0，这是因为此时两个向量完全相关
    if len(inA) < 3 : return 1.0
    return 0.5+0.5*corrcoef(inA, inB, rowvar = 0)[0][1]

def cosSim(inA, inB) :
    num = float(inA.T*inB)
    denom = la.norm(inA)*la.norm(inB)
    return 0.5+0.5*(num/denom)

# 基于SVD的评分估计
# 在recommend()中，svdEst用户替换对standEst()的调用，该函数对给定用户物品构建一个评分估计值。
# 与standEst()非常相似，不同之处就在于它在第64行对数据集进行了SVD分解。在SVD分解后，只利用包含
# 90%能量值的奇异值，这些奇异值以Numpy数组的形式得以保存。
def svdEst(dataMat, user, simMeas, item) :
    n = shape(dataMat)[1]
    simTotal = 0.0; ratSimTotal = 0.0
    U,Sigma,VT = la.svd(dataMat)
    # 使用奇异值构建一个对角矩阵
    Sig4 = mat(eye(65)*Sigma[:65])
    # 利用U矩阵将物品转换到低维空间中
    xformedItems = dataMat.T * U[:, :65] * Sig4.I
    # 对于给定的用户，for循环在用户对应行的所有元素上进行遍历，与standEst()函数中的for循环目的一样
    # 不同的是，这里的相似度是在低维空间下进行的。相似度的计算方法也会作为一个参数传递给该函数
    for j in range(n) :
        userRating = dataMat[user,j]
        if userRating == 0 or j == item : continue
        similarity = simMeas(xformedItems[item, :].T, xformedItems[j, :].T)
        # print便于了解相似度计算的进展情况
        print ('the %d and %d similarity is : %f' % (item, j, similarity))
        # 对相似度求和
        simTotal += similarity
        # 对相似度及评分值的乘积求和
        ratSimTotal += similarity * userRating
    if simTotal == 0 : return 0
    else : return ratSimTotal/simTotal

# 推荐引擎，会调用svdEst()函数，产生最高的N个推荐结果。
# simMeas：相似度计算方法
# estMethod：估计方法
def recommend(dataMat, user, N=10, simMeas=cosSim, estMethod=svdEst) :
    # 寻找未评级的物品，对给定用户建立一个未评分的物品列表
    unratedItems = nonzero(dataMat[user, :].A==0)[1]
    # 如果不存在未评分物品，退出函数，否则在所有未评分物品上进行循环
    if len(unratedItems) == 0 : return 'you rated everything'
    itemScores = []
    for item in unratedItems :
        # 对于每个未评分物品，通过调用svdEst()来产生该物品的预测评分。
        estimatedScore = estMethod(dataMat, user, simMeas, item)
        # 该物品的编号和估计得分值会放在一个元素列表itemScores
        itemScores.append((item, estimatedScore))
    # 寻找前N个未评级物品
    return  sorted(itemScores, key=lambda jj : jj[1], reverse=True)[:N]

In [ ]:
recommend(res,100)